In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import numpy as np
import pandas as pd
import json
import re
import shinra_util as util

In [3]:
with open("../data/compound_train.json") as f:
    train = json.load(f)['entry']

In [34]:
train_production_dict = {}
for entry in train:
    if len(entry['Attributes']['製造方法']) is 0: continue
    train_production_dict[str(entry['WikipediaID'])] = util.flatten([util.text2sentence(prod) for prod in entry['Attributes']['製造方法']])

In [4]:
wiki_data = util.read_jasonl("../data/jawiki-cirrussearch-dump_of_Compound.jsonl")

In [18]:
wiki_sentence_df = pd.DataFrame()
for entry in wiki_data:
    wiki_sentence_df = \
    wiki_sentence_df.append(
        pd.DataFrame({
            '_id': entry['index']['_id']
            , 'sentence': util.text2sentence(entry['text'])
        })
    )

## Trainデータ作成

In [19]:
train_id = [str(entry['WikipediaID']) for entry in train]

In [101]:
train_sentence_df = wiki_sentence_df.loc[wiki_sentence_df._id.isin(train_id)].reset_index(drop=True).assign(label = False)

In [106]:
for _id, productions in train_production_dict.items():
    train_sentence_df.loc[train_sentence_df._id == _id, 'label'] = \
        train_sentence_df.loc[train_sentence_df._id == _id].sentence.str.contains(util.contains_patt(productions))